In [1]:
import csv
import re
from sklearn.externals import joblib
# tag_count = joblib.load("/media/data/hotels/kdd11/processed/extracted_tag/kdd11_tag_count.pkl")
# tag_dict = joblib.load("/media/data/hotels/kdd11/processed/extracted_tag/kdd11_tag_dict.pkl")
tag_count, tag_dict = joblib.load('/media/data/hotels/kdd11/processed/extracted_tag/kdd11_tag_count_dict_c5.pkl')

In [2]:
def pop_count(tag_count, tag_dict):
    print(len(tag_count), len(tag_dict))
    pos = []
    for k, v in tag_count.most_common():
        if v <= 5:
            pos.append(k)

    for k in pos:
        tag_count.pop(k)
        tag_dict.pop(k)
    print(len(tag_count), len(tag_dict))

    joblib.dump([tag_count, tag_dict], "/media/data/hotels/kdd11/processed/extracted_tag/kdd11_tag_count_dict_c5.pkl")

In [20]:
dt_tags = []
for tag, count in tag_count.most_common():
    if re.match(r"\('DT', 'NN',", tag) and "JJ" in tag and count >= 1000:
        dt_tags.append(tag)

In [21]:
dt_tags

["('DT', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'JJ', 'CC', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'RB', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'IN', 'DT', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBZ', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'JJ', 'CC', 'DT', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'RB', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'DT', 'JJ', 'NN', '.')",
 "('DT', 'NN', 'VBD', 'JJ', ',', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'DT', 'J

In [24]:
interest_anpos = [("('DT', 'NN', 'VBD', 'JJ', '.')", (1, 3)),   
                  ("('JJ', 'NN', '.')", (1, 0)),
                  ("('DT', 'NN', 'VBZ', 'JJ', '.')", (1, 3)),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', '.')", (1, 4)),
                  ("('DT', 'NN', 'VBD', 'JJ', 'CC', 'JJ', '.')", (1, [3, 5])),
                  ("('NN', 'VBD', 'JJ', '.')", (0, 2)),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'JJ', '.')", (1, [4, 6])),
                  ("('DT', 'NN', 'VBZ', 'RB', 'JJ', '.')", (1, 4)),
                  ("('PRP', 'VBD', 'DT', 'JJ', 'NN', '.')", (4, 3)),
                  ("('RB', 'JJ', 'NN', '.')", (2, 1)),
                  ("('NN', 'VBD', 'JJ', 'CC', 'JJ', '.')", (0, [2, 4])),
                  ("('DT', 'NNS', 'VBD', 'JJ', '.')", (1, 3)),
                  ("('NN', 'VBD', 'RB', 'JJ', '.')", (0, 3))]

In [25]:
from collections import Counter

def write_csv_lines(filename, data):
    with open(filename, "w") as f:
        csvwriter = csv.writer(f)
        for line in data:
            csvwriter.writerow(line)
            
def location(tokens, loc):
    d = []
    if type(loc) is tuple:
        tok = [wd for wd in tokens[loc[0]:loc[1]+1] if re.sub(r'[^0-9a-zA-Z ]+', '', wd).isalnum()]
        if len(tok) > 0:
            d.append(" ".join(tok))
    elif type(loc) is list:
        tok = [tokens[l] for l in loc if re.sub(r'[^0-9a-zA-Z ]+', '', tokens[l]).isalnum()]
        if len(tok) > 0:
            d.extend(tok)
    else:
        if re.sub(r'[^0-9a-zA-Z ]+', '', tokens[loc]).isalnum():
            d.append(tokens[loc])
    return d


def extract_jj_nn(interest_anpos, tag_dict):
    noun_dict = Counter()
    adj_dict = Counter()
    for anpos in interest_anpos:
        pos, loc = anpos
        nloc, aloc = loc
        for sent in tag_dict[pos]:
            tokens = sent.lower().split()
            nouns = location(tokens, nloc)
#             if "one" in nouns:
#                 print(pos)
#                 print(sent)
            noun_dict.update(nouns)
            adjs = location(tokens, aloc)
            adj_dict.update(adjs)
    write_csv_lines("/media/data/hotels/kdd11/processed/extracted_tag/tag_aspects_dt.csv", noun_dict.most_common())
    return noun_dict, adj_dict

In [26]:
dt_tags

["('DT', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'JJ', 'CC', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'RB', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'IN', 'DT', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBZ', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'JJ', 'CC', 'DT', 'NN', 'VBD', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'RB', 'RB', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBZ', 'DT', 'JJ', 'NN', '.')",
 "('DT', 'NN', 'VBD', 'JJ', ',', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'JJ', '.')",
 "('DT', 'NN', 'VBD', 'DT', 'J

In [31]:
tag_dict["('DT', 'NN', 'VBD', 'JJ', 'CC', 'RB', 'VBN', '.')"]

['The room was spacious and well decorated .',
 'The room was spacious and well maintained .',
 'The food was nice and delicately presented .',
 'The room was ornate but not overdone .',
 'The room was comfortable and nicely appointed .',
 'The hotel was clean and well maintained .',
 'The room was large and well decorated .',
 'The pool was huge and never crowded .',
 'The room was spacious and well appointed .',
 'The bathroom was clean and nicely appointed .',
 'The hotel was clean and well decorated .',
 'The food was good and reasonably priced .',
 'The resort was lush and well maintained .',
 'The beach was beautiful but quite crowded .',
 'The bathroom was small but well appointed .',
 'The food was delicious and reasonably priced .',
 'The hotel was clean and well run .',
 'The room was spacious and nicely docorated .',
 'The hotel was clean and nicely decorated .',
 'The hotel was clean and well decorated .',
 'The casino was super-clean and nicely arranged .',
 'The bathroom 

In [37]:
interest_anpos = [("('DT', 'NN', 'VBD', 'JJ', '.')", (1, 3)),
                  ("('DT', 'NN', 'VBZ', 'JJ', '.')", (1, 3)),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', '.')", (1, 4)),
                  ("('DT', 'NN', 'VBD', 'JJ', 'CC', 'JJ', '.')", (1, [3, 5])),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'JJ', '.')", (1, [4, 6])),
                  ("('DT', 'NN', 'VBZ', 'RB', 'JJ', '.')", (1, 4)),
                  ("('DT', 'NN', 'NN', 'VBD', 'JJ', '.')", ((1, 2), 4)),
                  ("('DT', 'NN', 'VBZ', 'JJ', 'CC', 'JJ', '.')", (1, [3, 5])),
                  ("('DT', 'NN', 'VBD', 'JJ', 'CC', 'RB', 'JJ', '.')", (1, [3, 6])),
                  ("('DT', 'NN', 'VBZ', 'RB', 'JJ', 'CC', 'JJ', '.')", (1, [4, 6])),
                  ("('DT', 'NN', 'IN', 'DT', 'NN', 'VBD', 'JJ', '.')", ((1, 4), 6)),
                  ("('DT', 'NN', 'NN', 'VBD', 'RB', 'JJ', '.')", ((1, 2), 5)),
                  ("('DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'JJ', '.')", ((1,4), 6)),
                  ("('DT', 'NN', 'NN', 'VBZ', 'JJ', '.')", ((1, 2), 4)),
                  ("('DT', 'NN', 'VBD', 'JJ', 'CC', 'DT', 'NN', 'VBD', 'JJ', '.')", ([1, 6], [3, 8])),
                  ("('DT', 'NN', 'VBD', 'RB', 'RB', 'JJ', '.')", (1, 5)),
                  ("('DT', 'NN', 'NN', 'VBD', 'JJ', 'CC', 'JJ', '.')", ((1, 2), [4, 6])),
                  ("('DT', 'NN', 'VBZ', 'DT', 'JJ', 'NN', '.')", (1, 4)),
                  ("('DT', 'NN', 'VBD', 'JJ', ',', 'JJ', 'CC', 'JJ', '.')", (1, [3, 5, 7])),
                  ("('DT', 'NN', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'JJ', '.')", ((1, 2), [5, 7])),
                  ("('DT', 'NN', 'VBD', 'DT', 'JJ', 'NN', '.')", (1, 4)),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'RB', 'JJ', '.')", (1, [4, 7])),
                  ("('DT', 'NN', 'VBZ', 'IN', 'DT', 'JJ', 'NN', '.')", (1, 5)),
                  ("('DT', 'NN', 'CC', 'NN', 'VBD', 'JJ', '.')", ([1, 3], 5)),
                  ("('DT', 'NN', 'VBD', 'JJ', 'RB', '.')", (1, 3)),
                  ("('DT', 'NN', 'NN', 'VBZ', 'RB', 'JJ', '.')", ((1, 2), 5)),
                  ("('DT', 'NN', 'VBZ', 'JJ', 'CC', 'RB', 'JJ', '.')", (1, [3, 6])),
                  ("('DT', 'NN', 'VBD', 'JJ', 'CC', 'RB', 'VBN', '.')", (1, 3)),
                  ("('DT', 'NN', 'IN', 'DT', 'NN', 'VBD', 'RB', 'JJ', '.')", ((1, 4), 7)),
                  ("('DT', 'NN', 'PRP', 'VBD', 'JJ', '.')", (1, 4)),
                  ("('DT', 'NN', 'VBZ', 'RB', 'RB', 'JJ', '.')", (1, 5)),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'DT', 'NN', 'VBD', 'JJ', '.')", ([1, 7], [4, 9])),
                  ("('DT', 'NN', 'VBP', 'RB', 'JJ', 'CC', 'JJ', '.')", (1, [4, 6])),
                  ("('DT', 'NN', 'VBD', 'JJ', 'CC', 'DT', 'NN', 'VBD', 'RB', 'JJ', '.')", ([1, 6], [3, 9])),
                  ("('DT', 'NN', 'VBD', 'JJ', 'NN')", (1, 3)),
                  ("('DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'JJ', '.')", ((1, 4), 7)),
                  ("('DT', 'NN', 'NNS', 'VBD', 'JJ', '.')", ((1, 2), 4)),
                  ("('DT', 'NN', 'PRP', 'VBZ', 'JJ', '.')", (1, 4)),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', ',', 'JJ', 'CC', 'JJ', '.')", (1, [4, 6, 8])),
                  ("('DT', 'NN', 'VBD', 'RB', 'JJ', 'CC', 'VBG', '.')", (1, 4))]
     
noun_dict, adj_dict = extract_jj_nn(interest_anpos, tag_dict)

In [38]:
noun_dict.most_common(10)

[('staff', 44846),
 ('room', 30942),
 ('hotel', 24273),
 ('location', 15283),
 ('bed', 9168),
 ('bathroom', 8875),
 ('service', 8715),
 ('food', 7303),
 ('breakfast', 5700),
 ('pool', 4668)]

In [9]:
def read_csv(fname):
    data = []
    with open(fname, "r") as f:
        csvreader = csv.reader(f)
        for line in csvreader:
            data.append(line)
    return data

In [19]:
pos_data = read_csv(pos_file)
sbut_clauses = [line[0] for line in pos_data if re.match(r"but", line[0])]
ibut_clauses = [line[0] for line in pos_data if not re.match(r"but", line[0]) and line[1]=="but"]